<a href="https://colab.research.google.com/github/hvarS/CS60075-Team28-Task-1/blob/main/attention_multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # get extra dependencies
 
!pip install transformers

     |████████████████████████████████| 2.2MB 17.2MB/s 
     |████████████████████████████████| 3.3MB 49.4MB/s 
     |████████████████████████████████| 870kB 50.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=dbcf8757e54436843db65cc61c8a57b80a889f4787a51e28831d92ca0084e9c0
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d embeddings
!rm glove.6B.zip
!rm embeddings/glove.6B.50d.txt
!rm embeddings/glove.6B.100d.txt
!rm embeddings/glove.6B.200d.txt

--2021-04-12 11:42:22--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-12 11:42:22--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-12 11:42:22--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Importing

In [ ]:
import os
import pandas as pd
import time
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from transformers import BertTokenizer, TFBertModel
from collections import namedtuple
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding

In [ ]:
FOLDER_PATH = "/content/drive/MyDrive/CS60075-Team28-Task-1/"
DATA_FOLDER = os.path.join(FOLDER_PATH,"data/preprocessed")

In [ ]:
# import evaluate function
import sys
sys.path.append(FOLDER_PATH)
from eval import evaluate

In [ ]:
data = pd.read_csv(os.path.join(DATA_FOLDER, "lcp_single_train_preprocessed.csv"), index_col=0)
data['token'] = data['token'].astype(str)
data['sentence'] = data['sentence'].astype(str)

data.head()

,corpus,sentence,token,complexity
id,,,,
3ZLW647WALVGE8EBR50EGUBPU4P32A,bible,behold came river seven cattle sleek fat fed m...,river,0.000000
34R0BODSP1ZBN3DVY8J8XSIY551E5C,bible,fellow bondservant brother prophet keep word book,brother,0.000000
3S1WOPCJFGTJU2SGNAN2Y213N6WJE3,bible,man lord land said u know honest men leave one...,brother,0.050000
3BFNCI9LYKQN09BHXHH9CLSX5KP738,bible,shimei sixteen son six daughter brother didnt ...,brother,0.150000
3G5RUKN2EC3YIWSKUXZ8ZVH95R49N2,bible,put brother far,brother,0.263889


In [ ]:
data_multi = pd.read_csv(os.path.join(DATA_FOLDER, "lcp_multi_train_preprocessed.csv"), index_col=0)
data_multi['token'] = data_multi['token'].astype(str)
data_multi['sentence'] = data_multi['sentence'].astype(str)

data_multi.head()

,corpus,sentence,token,complexity
id,,,,
3S37Y8CWI80N8KVM53U4E6JKCDC4WE,bible,seventh day sabbath yahweh god shall work son ...,seventh day,0.027778
3WGCNLZJKF877FYC1Q6COKNWTDWD11,bible,let man test own work take pride neighbor,own work,0.050000
3UOMW19E6D6WQ5TH2HDD74IVKTP5CB,bible,understanding made heaven loving kindness endu...,loving kindness,0.050000
36JW4WBR06KF9AXMUL4N476OMF8FHD,bible,remember god also spare according greatness lo...,loving kindness,0.050000
3HRWUH63QU2FH9Q8R7MRNFC7JX2N5A,bible,loving kindness better life lip shall praise,loving kindness,0.075000


In [ ]:
data = pd.concat([data, data_multi])
print(len(data))

9179


## Features - load data 

In [ ]:
f1 = pd.read_csv(os.path.join(FOLDER_PATH, "data/extra_features/lcp_single_train_features.csv"), index_col=0)
f1['token'] = f1['token'].astype(str)
f1['sentence'] = f1['sentence'].astype(str)
f1.set_index("id", inplace=True)

# drop unwanted features
f1.drop(['parse', 'lemma'], axis=1, inplace=True)

print(f1.columns)

f2 = pd.read_csv(os.path.join(FOLDER_PATH, "data/added_corpus_presence/lcp_single_train_preprocessed.csv"), index_col=0)
f2['token'] = f2['token'].astype(str)
f2['sentence'] = f2['sentence'].astype(str)
print(f2.columns)

features = f1.merge(f2, on=['id','sentence', 'corpus', 'token', 'complexity'])

Index(['sentence', 'corpus', 'token', 'complexity', 'token_length',
       'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms',
       'google frequency'],
      dtype='object')
Index(['corpus', 'sentence', 'token', 'complexity', 'biomedical', 'bible',
       'subtitles', 'wiki', 'familarity'],
      dtype='object')


In [ ]:
# fill pos nan by NN, as they are in majority
features['pos'] = features['pos'].fillna('NN')
features['token_length'] = features['token_length'].fillna(0)

# categorical encoding
labels = dict(features['pos'].value_counts())
labels = { k:i for i,k in enumerate(labels)}
labels['POS'] = len(labels)
print(labels)

features['pos'] = features['pos'].apply((lambda x: labels[x]))

def get_vowels(word):
    val = 0
    for w in word:
        if(w in ['A', 'a', 'E', 'e', 'I', 'i', 'O', 'o', 'U','u']):
            val+=1
    return val

features['token_vowels'] = features['token'].apply(lambda x: get_vowels(x) )


# scaler = preprocessing.StandardScaler()
# features[['token_length', 'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency', 'familarity', 'token_vowels']] =  \
#     scaler.fit_transform(features[['token_length', 'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency', 'familarity', 'token_vowels']])

features.head()

{'NN': 0, 'JJ': 1, 'NNP': 2, 'NNS': 3, 'VBG': 4, 'VB': 5, 'RB': 6, 'VBP': 7, 'UH': 8, 'CD': 9, 'VBN': 10, 'FW': 11, 'VBZ': 12, 'VBD': 13, 'IN': 14, 'JJR': 15, 'GW': 16, 'PRP': 17, 'MD': 18, 'LS': 19, 'WRB': 20, 'JJS': 21, 'AFX': 22, 'POS': 23}


,sentence,corpus,token,complexity,token_length,syllables,pos,dep num,synonyms,hypernyms,hyponyms,google frequency,biomedical,bible,subtitles,wiki,familarity,token_vowels
id,,,,,,,,,,,,,,,,,,
3ZLW647WALVGE8EBR50EGUBPU4P32A,behold came river seven cattle sleek fat fed m...,bible,river,0.000000,5.0,2,0,0,1,1,0,173.485953,1,0,0,0,565,2
34R0BODSP1ZBN3DVY8J8XSIY551E5C,fellow bondservant brother prophet keep word book,bible,brother,0.000000,7.0,2,0,3,5,1,3,112.198857,1,0,0,0,598,2
3S1WOPCJFGTJU2SGNAN2Y213N6WJE3,man lord land said u know honest men leave one...,bible,brother,0.050000,7.0,2,0,1,5,1,3,112.198857,1,0,0,0,598,2
3BFNCI9LYKQN09BHXHH9CLSX5KP738,shimei sixteen son six daughter brother didnt ...,bible,brother,0.150000,7.0,2,0,4,5,1,3,112.198857,1,0,0,0,598,2
3G5RUKN2EC3YIWSKUXZ8ZVH95R49N2,put brother far,bible,brother,0.263889,7.0,2,0,2,5,1,3,112.198857,1,0,0,0,598,2


In [ ]:
multi_f1 = pd.read_csv(os.path.join(FOLDER_PATH, "data/extra_features/lcp_multi_train_split_features.csv"), index_col=0)
multi_f1['token'] = multi_f1['token'].astype(str)
multi_f1['sentence'] = multi_f1['sentence'].astype(str)
multi_f1.set_index("id", inplace=True)

# drop unwanted features
multi_f1.drop(['parse', 'token1', 'token2', 'lemma1', 'lemma2', 'Unnamed: 0.1'], axis=1, inplace=True)

multi_f2 = pd.read_csv(os.path.join(FOLDER_PATH, "data/added_corpus_presence/lcp_multi_train_preprocessed.csv"), index_col=0)
multi_f2['token'] = multi_f2['token'].astype(str)
multi_f2['sentence'] = multi_f2['sentence'].astype(str)

multi_features = multi_f1.merge(multi_f2, on=['id','sentence', 'corpus', 'token', 'complexity'])
multi_features.head(2)

,sentence,corpus,token,complexity,token_length,syllables,pos1,dep num1,pos2,dep num2,synonyms1,hypernyms1,hyponyms1,synonyms2,hypernyms2,hyponyms2,google frequency1,google frequency2,token_vowels,biomedical,bible,subtitles,wiki,familarity
id,,,,,,,,,,,,,,,,,,,,,,,,
3S37Y8CWI80N8KVM53U4E6JKCDC4WE,seventh day sabbath yahweh god shall work son ...,bible,seventh day,0.027778,11,3,JJ,0,NN,1,4,1,0,10,1,7,24.522564,682.298213,3,1,0,1,0,297.5
3WGCNLZJKF877FYC1Q6COKNWTDWD11,let man test own work take pride neighbor,bible,own work,0.050000,8,2,JJ,0,NN,3,2,0,1,34,1,27,0.000000,1022.711588,2,1,1,1,1,600.5


In [ ]:
# fill pos nan by NN, as they are in majority
multi_features['pos2'] = multi_features['pos2'].fillna('NN')

multi_features['pos1'] = multi_features['pos1'].apply((lambda x: labels[x]))
multi_features['pos2'] = multi_features['pos2'].apply((lambda x: labels[x]))

# scaler = preprocessing.StandardScaler()
# multi_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']] =  \
#     scaler.fit_transform(multi_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']])


multi_features.head()

,sentence,corpus,token,complexity,token_length,syllables,pos1,dep num1,pos2,dep num2,synonyms1,hypernyms1,hyponyms1,synonyms2,hypernyms2,hyponyms2,google frequency1,google frequency2,token_vowels,biomedical,bible,subtitles,wiki,familarity
id,,,,,,,,,,,,,,,,,,,,,,,,
3S37Y8CWI80N8KVM53U4E6JKCDC4WE,seventh day sabbath yahweh god shall work son ...,bible,seventh day,0.027778,11,3,1,0,0,1,4,1,0,10,1,7,24.522564,682.298213,3,1,0,1,0,297.5
3WGCNLZJKF877FYC1Q6COKNWTDWD11,let man test own work take pride neighbor,bible,own work,0.050000,8,2,1,0,0,3,2,0,1,34,1,27,0.000000,1022.711588,2,1,1,1,1,600.5
3UOMW19E6D6WQ5TH2HDD74IVKTP5CB,understanding made heaven loving kindness endu...,bible,loving kindness,0.050000,15,4,4,0,0,2,10,1,12,3,1,4,339.132903,14.221491,4,1,1,1,1,566.5
36JW4WBR06KF9AXMUL4N476OMF8FHD,remember god also spare according greatness lo...,bible,loving kindness,0.050000,15,4,4,1,0,2,10,1,12,3,1,4,339.132903,14.221491,4,1,1,1,1,566.5
3HRWUH63QU2FH9Q8R7MRNFC7JX2N5A,loving kindness better life lip shall praise,bible,loving kindness,0.075000,15,4,4,0,0,0,10,1,12,3,1,4,339.132903,14.221491,4,1,1,1,1,566.5


In [ ]:
# merge both single and multi features

features['pos1'] = features['pos'].copy()
features['pos2'] = features['pos'] 

features['dep num1'] = features['dep num'] 
features['dep num2'] = features['dep num'] 

features['synonyms1'] = features['synonyms'] 
features['synonyms2'] = features['synonyms'] 

features['hypernyms1'] = features['hypernyms'] 
features['hypernyms2'] = features['hypernyms'] 

features['hyponyms1'] = features['hyponyms'] 
features['hyponyms2'] = features['hyponyms'] 

features['google frequency1'] = features['google frequency'] 
features['google frequency2'] = features['google frequency1'] 

features.drop(['pos','dep num', 'synonyms', 'hyponyms', 'hypernyms', 'google frequency'], axis=1, inplace=True)

features = features.append( multi_features)
print(len(features))

scaler = preprocessing.StandardScaler()
features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']] =  \
    scaler.fit_transform(features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']])

features.head()

9179


,sentence,corpus,token,complexity,token_length,syllables,biomedical,bible,subtitles,wiki,familarity,token_vowels,pos1,pos2,dep num1,dep num2,synonyms1,synonyms2,hypernyms1,hypernyms2,hyponyms1,hyponyms2,google frequency1,google frequency2
id,,,,,,,,,,,,,,,,,,,,,,,,
3ZLW647WALVGE8EBR50EGUBPU4P32A,behold came river seven cattle sleek fat fed m...,bible,river,0.000000,-0.797042,-0.505404,1,0,0,0,1.276865,-0.689858,-0.323134,-0.249326,-0.707506,-0.831106,-0.729400,-0.754824,0.336733,0.195761,-0.333525,-0.369612,0.419993,0.466245
34R0BODSP1ZBN3DVY8J8XSIY551E5C,fellow bondservant brother prophet keep word book,bible,brother,0.000000,-0.337058,-0.505404,1,0,0,0,1.405225,-0.689858,-0.323134,-0.249326,1.709990,1.365895,-0.024553,-0.063647,0.336733,0.195761,-0.168281,-0.211593,0.088143,0.097424
3S1WOPCJFGTJU2SGNAN2Y213N6WJE3,man lord land said u know honest men leave one...,bible,brother,0.050000,-0.337058,-0.505404,1,0,0,0,1.405225,-0.689858,-0.323134,-0.249326,0.098326,-0.098772,-0.024553,-0.063647,0.336733,0.195761,-0.168281,-0.211593,0.088143,0.097424
3BFNCI9LYKQN09BHXHH9CLSX5KP738,shimei sixteen son six daughter brother didnt ...,bible,brother,0.150000,-0.337058,-0.505404,1,0,0,0,1.405225,-0.689858,-0.323134,-0.249326,2.515822,2.098228,-0.024553,-0.063647,0.336733,0.195761,-0.168281,-0.211593,0.088143,0.097424
3G5RUKN2EC3YIWSKUXZ8ZVH95R49N2,put brother far,bible,brother,0.263889,-0.337058,-0.505404,1,0,0,0,1.405225,-0.689858,-0.323134,-0.249326,0.904158,0.633561,-0.024553,-0.063647,0.336733,0.195761,-0.168281,-0.211593,0.088143,0.097424


## Splitting the Dataset into Test/Train

In [ ]:
train, test = train_test_split(features, test_size=0.1, random_state=12)
train.head()

,sentence,corpus,token,complexity,token_length,syllables,biomedical,bible,subtitles,wiki,familarity,token_vowels,pos1,pos2,dep num1,dep num2,synonyms1,synonyms2,hypernyms1,hypernyms2,hyponyms1,hyponyms2,google frequency1,google frequency2
id,,,,,,,,,,,,,,,,,,,,,,,,
3BAKUKE49HC18PHHJR1WT9408CA1RE,gathered men became captain troop david killed...,bible,men,0.156250,-1.257027,-1.105235,1,0,0,0,1.494688,-1.231397,1.480666,1.738998,0.098326,-0.098772,1.208928,1.145912,0.336733,0.195761,-0.278444,-0.316939,2.790531,3.100880
3P7RGTLO6EDBF9HMPQLS3YBPHGYKA8,seeing divine power granted u thing pertain li...,bible,virtue,0.303571,-0.567050,-0.505404,1,1,1,0,0.903456,-0.148320,-0.323134,-0.249326,2.515822,2.098228,-0.200765,-0.236441,0.336733,0.195761,-0.333525,-0.369612,-0.279144,-0.310782
3UY4PIS8QRLGTRPPQKKFKPZQ7HS1N1,thus say yahweh army god israel dont let proph...,bible,prophet,0.296875,-0.337058,-0.505404,1,0,1,0,1.035705,-0.689858,-0.323134,-0.249326,-0.707506,-0.831106,-0.553188,-0.582030,0.336733,0.195761,-0.168281,-0.211593,-0.396182,-0.440858
3VADEH0UHCXGQW0WJICRKFRUXY9PSC,e mr president spanish member resident canary ...,europarl,victim,0.191176,-0.567050,-0.505404,1,1,1,0,1.066822,-0.689858,-0.323134,-0.249326,-0.707506,-0.831106,-0.553188,-0.582030,0.336733,0.195761,0.272371,0.209791,-0.335287,-0.373179
3VGZ74AYTGGU62I1JDW4B19KA2MCGJ,subsequent transgenedriven reintroduction glur...,biomed,hippocampus,0.531250,0.582911,0.694260,1,1,1,1,-0.920806,0.393219,-0.323134,-0.249326,0.904158,0.633561,-0.553188,-0.582030,0.336733,0.195761,-0.333525,-0.369612,-0.502271,-0.558767


In [ ]:
sentences_train_list = list(train['sentence'])
complexity_train_list = list(train['complexity'])
tokens_train_list = list(train['token'])

sentences_test_list = list(test['sentence'])
complexity_test_list = list(test['complexity'])
tokens_test_list = list(test['token'])

## Getting embeddings (300 dimensional GloVe) and Preprocessing to I/P in Model

In [ ]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

  return word_to_vec_map

word_to_vec_map = read_glove_vector('embeddings/glove.6B.300d.txt')

# get embeddings, and pad till max_len
def get_embeddings(sentences, max_len=0):
    sent_emb = []
    for s in sentences:
        
        # fill unk by nan
        # calculate mean over non nan embeddings
        # fill unk by the mean embedding of sentence
        # pad 0 vectors till max_len

        temp_sent_emb = [ word_to_vec_map[x] if x in word_to_vec_map else np.full((300,), np.nan) for x in s.split() ]
        mean_emb = np.nanmean(np.array(temp_sent_emb), axis=0)

        temp_sent_emb = np.array([ mean_emb if np.isnan(x[0]) else x for x in temp_sent_emb ])
        temp_sent_emb = np.concatenate((temp_sent_emb, np.zeros((max_len-temp_sent_emb.shape[0],300))))

        sent_emb.append(temp_sent_emb)

    return np.array(sent_emb)

In [ ]:
# max len for the sentence used for padding
max_len_sent = max(max([len(s.split()) for s in sentences_train_list]) , max([len(s.split()) for s in sentences_test_list])) 
print(max_len_sent)

max_len_token = max(max([len(s.split()) for s in tokens_train_list]) , max([len(s.split()) for s in tokens_test_list])) 
print(max_len_token)

123
2


In [ ]:
train_sent_emb = get_embeddings(sentences_train_list, max_len_sent)
test_sent_emb = get_embeddings(sentences_test_list, max_len_sent)

print("sent emb shape train : {}".format(train_sent_emb.shape))
print("sent emb shape test : {}".format(test_sent_emb.shape))


# token_position_data
# for each token we should have [start index, lenght of token]

train_token_data =   [ 
                        [ 
                            len(s.split(t)[0].split()), 
                            len(t.split())
                        ]
                        for s,t in zip(sentences_train_list, tokens_train_list) 
                    ]

test_token_data =   [ 
                        [ 
                            len(s.split(t)[0].split()), 
                            len(t.split())
                        ]
                        for s,t in zip(sentences_test_list, tokens_test_list) 
                    ]

# features_train = train[['token_length', 'token_vowels', 'syllables', 'pos', 'dep num', 
#                         'synonyms', 'hypernyms', 'hyponyms', 
#                         'google frequency', 
#                         'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values

# features_test = test[['token_length', 'token_vowels', 'syllables', 'pos', 'dep num', 
#                         'synonyms', 'hypernyms', 'hyponyms', 
#                         'google frequency', 
#                         'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values


features_train = train[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
                        'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
                        'google frequency1', 'google frequency2', 
                        'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values

features_test = test[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
                        'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
                        'google frequency1', 'google frequency2', 
                        'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values

print("feature emb shape train : {}".format(features_train.shape))
print("feature emb shape test : {}".format(features_test.shape))

sent emb shape train : (8261, 123, 300)
sent emb shape test : (918, 123, 300)
feature emb shape train : (8261, 20)
feature emb shape test : (918, 20)




## Model Class - BiLSTM

In [ ]:
# -------------------------------------------------------------------------------------
# feed fwd neural network :
# 1 hidden size layer, 1 output_size layer  and a dropout between them
# used for calculating weights as well as final complexity prediction 
#-------------------------------------------------------------------------------------

class FFN(tf.keras.layers.Layer):
    def __init__(
        self, 
        hidden_size,
        output_size,
        rate,
        softmax=False,
    ):
        super(FFN, self).__init__()

        self.layer1 = tf.keras.layers.Dense(hidden_size, activation="relu")  # (batch_size, hidden_size)
        self.dropout = tf.keras.layers.Dropout(rate)
        self.layer2 = tf.keras.layers.Dense(output_size, activation= "softmax" if softmax is True else None)  # (batch_size, output_size)

    def call(self, x, training):
        return self.layer2(self.dropout(self.layer1(x), training=training) ) 
        # return self.layer2(self.dropout(x, training=training)) 
        
# -------------------------------------------------------------------------------------
# Custom Model for predicting complexity
#-------------------------------------------------------------------------------------

class OurModelBiLSTM(tf.keras.Model):
    def __init__(
        self,
        lstm_units,
        hidden_size,
        random_seed,
        seq_len,
        embedding_size,
        rate=0.25
        ):
        """
        hidden_size - for FFN
        """

        super(OurModelBiLSTM, self).__init__()

        tf.random.set_seed(random_seed)
        self.random_seed = random_seed
        self.lstm_units = lstm_units

        # self.dense1 = tf.keras.layers.Dense(2*self.lstm_units)
        self.bilstm = tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(self.lstm_units, return_sequences=True), input_shape=(seq_len,embedding_size)
            )
        # self.bilstm = tf.keras.layers.LSTM(self.lstm_units, return_sequences=True)
        self.dropout = tf.keras.layers.Dropout(rate=rate)

        self.layernorm = tf.keras.layers.LayerNormalization()
        self.dense = tf.keras.layers.Dense(2*self.lstm_units)
        self.getWeights = FFN(hidden_size, 3, rate, softmax=True)
        self.final_layer = FFN(hidden_size, 1, rate)

    
    # one fwd pass on the model
    def call(self, input_seq, token_position, feature_emb, training=False):
        """
        input_seq : [batch_size, seq_len, embedding_size]
        token_position = [batch_size, 2] - [start,length] for tokens for which complexity is to be predicted
        """

        # bi-lstm pass
        bilstm_output = self.bilstm(input_seq)

        # bilstm_output = self.bilstm(input_seq)      #(batch_size, seq_len, 2*lstm_units)
        bilstm_output = self.dropout(bilstm_output, training=training)

        # extract token embeddings    
        # for each example we have start token and length
        # we take mean of these embeddings position

        token_emb = tf.stack(
                [tf.reduce_mean(tf.gather(i, tf.range(j[0],j[0]+j[1]), axis=0), axis=0) for i,j in zip(bilstm_output,token_position) ]
            )   # (batch_size, 2*lstm_units)

        # combine all embeddings - take mean
        added_emb = tf.reduce_mean(bilstm_output, axis=1)    # (batch_size, 2*lstm_units)
        
        # feature_emb - (batch_shape, features)
        feature_emb = self.dense(feature_emb)           # (batch_size, 2*lstm_units)
        # feature_emb = tf.cast(feature_emb, tf.float32)

        # get weights
        # weights = self.getWeights(self.layernorm(tf.add_n([token_emb,added_emb, feature_emb])), training)    # (batch_size, 3)
        weights = self.getWeights(self.layernorm(tf.concat([token_emb,added_emb, feature_emb], axis=1)), training)    # (batch_size, 3)

        # generate attenton-based final embeddings
        # final_emb = weights[0]*token_emb + weights[1]*added_emb + weights[3]*features_emb

        final_emb = tf.zeros(shape=(), dtype=tf.dtypes.float32)   # (bacth_size, 2*lstm_units)
        final_emb += tf.expand_dims(weights[:, 0], axis=1) * token_emb
        final_emb += tf.expand_dims(weights[:, 1], axis=1) * added_emb
        final_emb += tf.expand_dims(weights[:, 2], axis=1) * feature_emb

        # output complexity
        final_output = self.final_layer(final_emb, training)

        return final_output

    # loss function
    def loss_function(self, real, pred):
        loss_ = tf.keras.losses.MSE(real, pred)
        l = tf.reduce_mean(loss_)
        return l

    # set optimizer
    def compile(self, optimizer):
        super(OurModelBiLSTM, self).compile()
        self.optimizer = optimizer


    # train step - fwd pass + back prop + update model weights
    # @tf.function() - may not work beacuse zip is used
    def train_step(self, input_seq, token_position, y, feature_emb):
        training=True
        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self(input_seq, token_position, feature_emb, training)  
            loss = self.loss_function(tf.reshape(y,(-1,1)), y_pred)

        # calculate gradients
        gradients = tape.gradient(loss, self.trainable_variables)

        # update model weights using gradients
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return loss

    # fit funtion of model
    def _fit(
        self, 
        input_seq_data, 
        token_position_data, 
        feature_emb_data,
        complexity, 
        epochs, 
        batch_size,
        val_input_seq_data=None,
        val_token_position_data=None,
        val_feature_emb_data=None,
        val_complexity=None
        ):

        # create tensorflow dataset
        tf_data = tf.data.Dataset.from_tensor_slices((input_seq_data, token_position_data, feature_emb_data, complexity))

        # shuffle and batch
        tf_data = tf_data.shuffle(100000, seed=self.random_seed).batch(batch_size)

        train_loss = tf.keras.metrics.Mean(name="train_loss")
        loss_train = {"MSE": []}
        loss_val = {"MSE": []}

        #training starts
        for epoch in range(epochs):
            start = time.time()
            
            # reset state for mean loss
            train_loss.reset_states()

            # loop over batches
            for step, x in tf_data.enumerate():

                # get inputs for model
                input_seq = x[0]
                token_position = x[1]
                feature_emb = x[2]
                y = x[3]

                # call trainstep
                loss = self.train_step(input_seq, token_position, y, feature_emb)
                train_loss(loss)

            loss_train["MSE"].append(train_loss.result().numpy())
            print("Epoch {} loss  MSE: {}, time taken: {:.2f}s".format(epoch + 1, loss_train["MSE"][-1], time.time() - start))

            # validation if provided
            if (val_input_seq_data is not None):
                assert val_token_position_data is not None and val_complexity is not None, "incorrect validation data"
                val_pred = self._predict(val_input_seq_data, val_token_position_data, val_feature_emb_data)
                loss_val["MSE"].append(self.loss_function(tf.reshape(val_complexity, (-1,1)), val_pred).numpy())
                print("Validation loss MSE : {}".format(loss_val["MSE"][-1]))
        
        if (val_input_seq_data is not None):
            return loss_train, loss_val
        return loss_train

    
    # predict function
    def _predict(self, input_seq_data, token_position_data, feature_emb_data):
        
        # create tensorflow dataset
        tf_data = tf.data.Dataset.from_tensor_slices((input_seq_data, token_position_data, feature_emb_data))

        # batch for memory constraints
        tf_data = tf_data.batch(512)

        pred_list = []
        for step, x in tf_data.enumerate():
            # get inputs for model
            input_seq = x[0]
            token_position = x[1]
            feature_emb = x[2]

            pred_list.append(self(input_seq, token_position, feature_emb))
        return tf.concat(pred_list, axis=0)

## Prepare test data features - only multi


In [ ]:
test_f1 = pd.read_csv(os.path.join(FOLDER_PATH, "data/extra_features/lcp_single_test_features.csv"), index_col=0)
test_f1['token'] = test_f1['token'].astype(str)
test_f1['sentence'] = test_f1['sentence'].astype(str)
test_f1.set_index("id", inplace=True)

# drop unwanted features
test_f1.drop(['parse', 'lemma'], axis=1, inplace=True)

print(test_f1.columns)

test_f2 = pd.read_csv(os.path.join(FOLDER_PATH, "data/added_corpus_presence/lcp_single_test_preprocessed.csv"), index_col=0)
test_f2['token'] = test_f2['token'].astype(str)
test_f2['sentence'] = test_f2['sentence'].astype(str)
print(test_f2.columns)

test_features = test_f1.merge(test_f2, on=['id','sentence', 'corpus', 'token'])

Index(['sentence', 'corpus', 'token', 'token_length', 'syllables', 'pos',
       'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency'],
      dtype='object')
Index(['corpus', 'sentence', 'token', 'biomedical', 'bible', 'subtitles',
       'wiki', 'familarity'],
      dtype='object')


In [ ]:
# fill pos nan by NN, as they are in majority
test_features['pos'] = test_features['pos'].fillna('NN')
test_features['token_length'] = test_features['token_length'].fillna(0)

test_features['pos'] = test_features['pos'].apply((lambda x: labels[x]))

def get_vowels(word):
    val = 0
    for w in word:
        if(w in ['A', 'a', 'E', 'e', 'I', 'i', 'O', 'o', 'U','u']):
            val+=1
    return val

test_features['token_vowels'] = test_features['token'].apply(lambda x: get_vowels(x) )


# test_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']] =  \
#     scaler.transform(test_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']])

# test_features[['token_length', 'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency', 'familarity', 'token_vowels']] = \
#     scaler.transform(test_features[['token_length', 'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency', 'familarity', 'token_vowels']])

test_features.head()

,sentence,corpus,token,token_length,syllables,pos,dep num,synonyms,hypernyms,hyponyms,google frequency,biomedical,bible,subtitles,wiki,familarity,token_vowels
id,,,,,,,,,,,,,,,,,
39TX062QX1OHFOH8FUL76K5L7D3X3S,speak much prince world come nothing,bible,prince,6,1,0,0,1,1,10,57.555460,1,0,0,0,506,2
3CIS7GGG65JS8I3AZ9RG54AE4MUUEA,house shall turned others field wife together ...,bible,inhabitant,10,4,0,0,1,1,30,1.907882,1,0,1,0,469,4
379OL9DBSSESUVWY1Z8JGBFG9BTY92,stranger terrible nation cut left mountain val...,bible,bough,5,1,0,2,1,1,0,1.513619,1,1,1,1,0,2
3DFYDSXB2W00JYP2DA272KN69UQUJV,sharpen tongue like sword aim arrow deadly word,bible,arrow,5,2,0,0,2,1,0,22.707380,0,0,1,0,490,2
31YWE12TE0CZG7IVH6OXJ1H1CFPX7X,obey leader submit watch behalf soul give acco...,bible,account,7,2,0,0,14,1,7,216.439551,1,1,1,0,523,3


In [ ]:
test_multi_f1 = pd.read_csv(os.path.join(FOLDER_PATH, "data/extra_features/lcp_multi_test_split_features.csv"), index_col=0)
test_multi_f1['token'] = test_multi_f1['token'].astype(str)
test_multi_f1['sentence'] = test_multi_f1['sentence'].astype(str)
test_multi_f1.set_index("id", inplace=True)

# drop unwanted features
test_multi_f1.drop(['parse', 'token1', 'token2', 'lemma1', 'lemma2', 'Unnamed: 0.1'], axis=1, inplace=True)

test_multi_f2 = pd.read_csv(os.path.join(FOLDER_PATH, "data/added_corpus_presence/lcp_multi_test_preprocessed.csv"), index_col=0)
test_multi_f2['token'] = test_multi_f2['token'].astype(str)
test_multi_f2['sentence'] = test_multi_f2['sentence'].astype(str)

test_multi_features = test_multi_f1.merge(test_multi_f2, on=['id','sentence', 'corpus', 'token'])
test_multi_features['token'] = test_multi_f2['token'].astype(str)
test_multi_features.head(2)

,sentence,corpus,token,token_length,syllables,pos1,dep num1,pos2,dep num2,synonyms1,hypernyms1,hyponyms1,synonyms2,hypernyms2,hyponyms2,google frequency1,google frequency2,token_vowels,biomedical,bible,subtitles,wiki,familarity
id,,,,,,,,,,,,,,,,,,,,,,,
3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL,come intending bring bound chief priest,bible,chief priest,12,2,JJ,0,NN,2,4,1,10,2,2,11,0.000000,39.121551,4,1,0,1,0,483.0
302U8RURJZ1WF35NXY44RD66WL4NVH,day lord take away beauty anklet headband cres...,bible,crescent necklace,17,4,NN,1,NN,2,2,1,0,1,1,3,4.830131,4.021996,5,1,1,1,1,268.0


In [ ]:
# fill pos nan by NN, as they are in majority
test_multi_features['pos2'] = test_multi_features['pos2'].fillna('NN')

test_multi_features['pos1'] = test_multi_features['pos1'].apply((lambda x: labels[x]))
test_multi_features['pos2'] = test_multi_features['pos2'].apply((lambda x: labels[x]))

test_multi_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']] =  \
    scaler.transform(test_multi_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']])


test_multi_features.head()

,sentence,corpus,token,token_length,syllables,pos1,dep num1,pos2,dep num2,synonyms1,hypernyms1,hyponyms1,synonyms2,hypernyms2,hyponyms2,google frequency1,google frequency2,token_vowels,biomedical,bible,subtitles,wiki,familarity
id,,,,,,,,,,,,,,,,,,,,,,,
3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL,come intending bring bound chief priest,bible,chief priest,0.812903,-0.505404,0.278133,-0.707506,-0.249326,0.633561,-0.200765,0.336733,0.217289,-0.582030,3.160928,0.209791,-0.519379,-0.342350,0.393219,1,0,1,0,0.957911
302U8RURJZ1WF35NXY44RD66WL4NVH,day lord take away beauty anklet headband cres...,bible,crescent necklace,1.962864,0.694260,-0.323134,0.098326,-0.249326,0.633561,-0.553188,0.336733,-0.333525,-0.754824,0.195761,-0.211593,-0.493225,-0.553577,0.934757,1,1,1,1,0.121629
3UDTAB6HH6ZVX00DTRXAOJLWX0B094,unclean shall take ash burning sin offering ru...,bible,sin offering,0.812903,0.694260,-0.323134,1.709990,2.401772,-0.098772,0.504082,0.336733,-0.278444,1.837089,0.195761,-0.369612,-0.238985,0.113318,0.393219,1,0,1,0,0.053560
3L2OEKSTW9ASGQDOW725GFK5P77Y8D,precious treasure oil dwelling wise foolish ma...,bible,precious treasure,1.962864,0.694260,0.278133,-0.707506,-0.249326,-0.098772,-0.024553,-2.087483,-0.333525,0.109147,0.195761,-0.158920,-0.393608,-0.489569,2.559373,1,0,1,1,0.076898
39N6W9XWRDN795J6F5ET8S13DQKYGT,long god shall adversary reproach,bible,adversary reproach,2.192856,1.893924,-0.323134,-0.707506,-0.249326,2.098228,-0.729400,0.336733,-0.113199,-0.409235,0.195761,-0.264266,-0.489295,-0.547988,1.476296,1,0,1,1,-0.920806


In [ ]:
# single_sent_emb = get_embeddings(list(test_features['sentence']), max_len_sent)
multi_sent_emb = get_embeddings(list(test_multi_features['sentence']), max_len_sent)

# print("single emb shape train : {}".format(single_sent_emb.shape))
print("multi emb shape test : {}".format(multi_sent_emb.shape))


# token_position_data
# for each token we should have [start index, lenght of token]

# single_token_data =   [ 
#                         [ 
#                             len(s.split(t)[0].split()), 
#                             len(t.split())
#                         ]
#                         for s,t in zip(list(test_features['sentence']), list(test_features['token'])) 
#                     ]

multi_token_data =   [ 
                        [ 
                            len(s.split(t)[0].split()), 
                            len(t.split())
                        ]
                        for s,t in zip(list(test_multi_features['sentence']), list(test_multi_features['token'])) 
                    ]

# features_single = test_features[['token_length', 'token_vowels', 'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency', 'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']]
# features_single = test_features[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
#                         'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
#                         'google frequency1', 'google frequency2', 
#                         'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values

features_multi = test_multi_features[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
                        'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
                        'google frequency1', 'google frequency2', 
                        'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values

# print("feature emb shape single : {}".format(features_single.shape))
print("feature emb shape multi : {}".format(features_multi.shape))

multi emb shape test : (184, 123, 300)
feature emb shape multi : (184, 20)


## Run model


In [ ]:
model = OurModelBiLSTM(lstm_units=15, hidden_size=10, random_seed=12, seq_len=max_len_sent, embedding_size=300, rate=0.4)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005))

loss = model._fit ( train_sent_emb, 
                    train_token_data,
                    feature_emb_data = features_train,
                    complexity = complexity_train_list,
                    epochs=14, 
                    batch_size=32,
                    val_input_seq_data=test_sent_emb,
                    val_token_position_data = test_token_data,
                    val_feature_emb_data = features_test,
                    val_complexity = complexity_test_list
                )

Epoch 1 loss  MSE: 0.023469505831599236, time taken: 34.45s
Validation loss MSE : 0.012678328901529312
Epoch 2 loss  MSE: 0.013689023442566395, time taken: 34.61s
Validation loss MSE : 0.010032896883785725
Epoch 3 loss  MSE: 0.011827686801552773, time taken: 35.22s
Validation loss MSE : 0.009277915582060814
Epoch 4 loss  MSE: 0.011137045919895172, time taken: 34.40s
Validation loss MSE : 0.008250661194324493
Epoch 5 loss  MSE: 0.010887444019317627, time taken: 34.78s
Validation loss MSE : 0.009242299944162369
Epoch 6 loss  MSE: 0.010068163275718689, time taken: 34.24s
Validation loss MSE : 0.008398651145398617
Epoch 7 loss  MSE: 0.009919965639710426, time taken: 34.10s
Validation loss MSE : 0.00913756899535656
Epoch 8 loss  MSE: 0.010019904002547264, time taken: 34.28s
Validation loss MSE : 0.008295852690935135
Epoch 9 loss  MSE: 0.009518742561340332, time taken: 34.60s
Validation loss MSE : 0.008844422176480293
Epoch 10 loss  MSE: 0.009391745552420616, time taken: 34.43s
Validation lo

In [ ]:
SUBMISSION_FOLDER = os.path.join(FOLDER_PATH,"predictions/attn_app/multi")

if( not os.path.exists(SUBMISSION_FOLDER)):
    os.makedirs(SUBMISSION_FOLDER)

# predict 
pred = model._predict(multi_sent_emb, multi_token_data, features_multi)
pred = pd.DataFrame({"ID":test_multi_features.index, "complexity":pred.numpy().reshape(-1,)})
pred.to_csv(SUBMISSION_FOLDER+"/app1.csv", index=False, header=False)


evaluate(SUBMISSION_FOLDER, FOLDER_PATH+"/references/lcp_multi_test_labelled_preprocessed.csv")


For file app1.csv
pearson  :  0.8284854541415717
spearman :  0.8194230415403599
mae      :  0.08488548649686772
mse      :  0.010865609933319113
r2       :  0.5498384036490054


In [ ]:
model.summary()

Model: "our_model_bi_lstm_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection multiple                  37920     
_________________________________________________________________
dropout_6 (Dropout)          multiple                  0         
_________________________________________________________________
layer_normalization_2 (Layer multiple                  180       
_________________________________________________________________
dense_10 (Dense)             multiple                  630       
_________________________________________________________________
ffn_4 (FFN)                  multiple                  943       
_________________________________________________________________
ffn_5 (FFN)                  multiple                  321       
Total params: 39,994
Trainable params: 39,994
Non-trainable params: 0
___________________________________________

## sample test bilstm code

In [ ]:
# ----------- test model output shape

sample_model = OurModelBiLSTM(lstm_units=2, hidden_size=2001, random_seed=12, seq_len=10, embedding_size=10)

#(bs, seq_len, embedding_shape)
input = tf.random.uniform((32, 10, 10), dtype=tf.float32, minval=0, maxval=200)

# (start, len)
token_position_data = tf.constant([(3,2) for _ in range(32)])

output = sample_model(input, token_position_data)
print(output.shape)

#  ----------- test fit and predict

sample_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

input_seq_data = tf.random.uniform((100, 10, 10), dtype=tf.float32, minval=0, maxval=200)
token_position_data = [(3,5) for _ in range(100)]
y = tf.random.uniform((100, 1), dtype=tf.float32, minval=0, maxval=1)

val_input_seq_data = tf.random.uniform((100, 10, 10), dtype=tf.float32, minval=0, maxval=200)
val_token_position_data = [(3,5) for _ in range(100)]
val_y = tf.random.uniform((100, 1), dtype=tf.float32, minval=0, maxval=1)

loss = sample_model._fit( 
    input_seq_data = input_seq_data, 
    token_position_data = token_position_data, 
    complexity = y, 
    epochs = 10, 
    batch_size = 32,
    val_input_seq_data = val_input_seq_data,
    val_token_position_data = val_token_position_data,
    val_complexity=val_y
    )


'\nsample_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))\n\ninput_seq_data = tf.random.uniform((100, 10, 10), dtype=tf.float32, minval=0, maxval=200)\ntoken_position_data = [(3,5) for _ in range(100)]\ny = tf.random.uniform((100, 1), dtype=tf.float32, minval=0, maxval=1)\n\nval_input_seq_data = tf.random.uniform((100, 10, 10), dtype=tf.float32, minval=0, maxval=200)\nval_token_position_data = [(3,5) for _ in range(100)]\nval_y = tf.random.uniform((100, 1), dtype=tf.float32, minval=0, maxval=1)\n\nloss = sample_model._fit( \n    input_seq_data = input_seq_data, \n    token_position_data = token_position_data, \n    complexity = y, \n    epochs = 10, \n    batch_size = 32,\n    val_input_seq_data = val_input_seq_data,\n    val_token_position_data = val_token_position_data,\n    val_complexity=val_y\n    )\n'

# Code for other experiment - with BERT

## BERT CODE 

In [ ]:
# -------------------------------------------------------------------------------------
# feed fwd neural network :
# 1 hidden size layer, 1 output_size layer  and a dropout between them
# used for calculating weights as well as final complexity prediction 
#-------------------------------------------------------------------------------------

class FFN(tf.keras.layers.Layer):
    def __init__(
        self, 
        hidden_size,
        output_size,
        rate,
        softmax=False,
    ):
        super(FFN, self).__init__()

        self.layer1 = tf.keras.layers.Dense(hidden_size, activation="relu")  # (batch_size, hidden_size)
        self.dropout = tf.keras.layers.Dropout(rate)
        self.layer2 = tf.keras.layers.Dense(output_size, activation= "softmax" if softmax is True else None)  # (batch_size, output_size)

    def call(self, x, training):
        return self.layer2(self.dropout(self.layer1(x), training=training) ) 
        # return self.layer2(self.dropout(x, training=training)) 
        
# -------------------------------------------------------------------------------------
# Custom Model for predicting complexity
#-------------------------------------------------------------------------------------

class OurModelBert(tf.keras.Model):
    def __init__(
        self,
        hidden_size,
        random_seed,
        rate=0.25
        ):
        """
        hidden_size - for FFN
        """

        super(OurModelBert, self).__init__()

        tf.random.set_seed(random_seed)
        self.random_seed = random_seed

        self.dense = tf.keras.layers.Dense(768, activation="relu")
        self.dropout = tf.keras.layers.Dropout(rate=rate)
        self.getWeights = FFN(hidden_size, 3, rate, softmax=True)
        self.final_layer = FFN(hidden_size, 1, rate)

        self.layernorm = tf.keras.layers.LayerNormalization()

    
    # one fwd pass on the model
    def call(self, cls_emb, token_position, feature_emb, token_emb, training=False):
    

        # feature_emb = tf.concat([feature_emb, token_position], axis=1)
        feature_emb = self.dropout(self.dense(feature_emb), training=training)

        # get weights
        weights = self.getWeights(self.layernorm(tf.add_n([cls_emb, token_emb, feature_emb])), training)    # (batch_size, 3)

        # generate attenton-based final embeddings
        # final_emb = weights[0]*token_emb + weights[1]*added_emb + weights[3]*features_emb

        final_emb = tf.zeros(shape=(), dtype=tf.dtypes.float32)   # (bacth_size, 2*lstm_units)
        final_emb += tf.expand_dims(weights[:, 0], axis=1) * cls_emb
        final_emb += tf.expand_dims(weights[:, 1], axis=1) * token_emb
        final_emb += tf.expand_dims(weights[:, 2], axis=1) * feature_emb

        # output complexity
        final_output = self.final_layer(final_emb, training)

        return final_output

    # loss function
    def loss_function(self, real, pred):
        loss_ = tf.keras.losses.MSE(real, pred)
        l = tf.reduce_mean(loss_)
        return l

    # set optimizer
    def compile(self, optimizer):
        super(OurModelBert, self).compile()
        self.optimizer = optimizer


    # train step - fwd pass + back prop + update model weights
    # @tf.function() - may not work beacuse zip is used
    def train_step(self, cls_emb, token_position, feature_emb, token_emb, y):
        training=True
        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self(cls_emb, token_position, feature_emb, token_emb, training)  
            loss = self.loss_function(tf.reshape(y,(-1,1)), y_pred)

        # calculate gradients
        gradients = tape.gradient(loss, self.trainable_variables)

        # update model weights using gradients
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return loss

    # fit funtion of model
    def _fit(
        self, 
        cls_emb, 
        token_emb,
        token_position_data, 
        feature_emb,
        complexity, 
        epochs, 
        batch_size,
        val_cls_emb=None, 
        val_token_emb=None,
        val_token_position_data=None, 
        val_feature_emb=None,
        val_complexity=None, 
        ):

        # create tensorflow dataset
        tf_data = tf.data.Dataset.from_tensor_slices((cls_emb, token_emb, token_position_data, feature_emb, complexity))

        # shuffle and batch
        tf_data = tf_data.shuffle(100000, seed=self.random_seed).batch(batch_size)

        train_loss = tf.keras.metrics.Mean(name="train_loss")
        loss_train = {"MSE": []}
        loss_val = {"MSE": []}

        #training starts
        for epoch in range(epochs):
            start = time.time()
            
            # reset state for mean loss
            train_loss.reset_states()

            # loop over batches
            for step, x in tf_data.enumerate():

                # get inputs for model
                cls_emb = x[0] 
                token_emb = x[1]
                token_position = x[2] 
                feature_emb = x[3]
                complexity = x[4]
                
                # call trainstep
                loss = self.train_step(cls_emb, token_position, feature_emb, token_emb, complexity)
                train_loss(loss)

            loss_train["MSE"].append(train_loss.result().numpy())
            print("Epoch {} loss  MSE: {}, time taken: {:.2f}s".format(epoch + 1, loss_train["MSE"][-1], time.time() - start))

            # validation if provided
            # if (val_input_seq_data is not None):
            #     assert val_token_position_data is not None and val_complexity is not None, "incorrect validation data"
            #     val_pred = self._predict(val_input_seq_data, val_token_position_data, val_feature_emb_data)
            #     loss_val["MSE"].append(self.loss_function(tf.reshape(val_complexity, (-1,1)), val_pred).numpy())
            #     print("Validation loss MSE : {}".format(loss_val["MSE"][-1]))
        
        if (val_input_seq_data is not None):
            return loss_train, loss_val
        return loss_train

    
    # predict function
    def _predict(self, input_seq_data, token_position_data, feature_emb_data):
        # create tensorflow dataset
        tf_data = tf.data.Dataset.from_tensor_slices((input_seq_data, token_position_data, feature_emb_data))

        # batch for memory constraints
        tf_data = tf_data.batch(512)

        pred_list = []
        for step, x in tf_data.enumerate():
            # get inputs for model
            input_seq = x[0]
            token_position = x[1]
            feature_emb = x[2]

            pred_list.append(self(input_seq, token_position, feature_emb))
        return tf.concat(pred_list, axis=0)

In [ ]:
model = OurModelBert(hidden_size=256, random_seed=12, rate=0.3)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005))

In [ ]:
features_train = features[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
                        'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
                        'google frequency1', 'google frequency2', 
                        'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values
features_train.shape

(9179, 20)

In [ ]:
loss = model._fit ( cls_emb, 
                    token_emb,
                    token_position_data, 
                    feature_emb=features_train,
                    complexity=features['complexity'], 
                    epochs=50, 
                    batch_size=32,
                )

Epoch 1 loss  MSE: 0.048082321882247925, time taken: 32.62s
Epoch 2 loss  MSE: 0.013169867917895317, time taken: 4.38s
Epoch 3 loss  MSE: 0.012237255461513996, time taken: 4.27s
Epoch 4 loss  MSE: 0.011963734403252602, time taken: 4.13s
Epoch 5 loss  MSE: 0.012074682861566544, time taken: 4.38s
Epoch 6 loss  MSE: 0.01182155404239893, time taken: 4.26s
Epoch 7 loss  MSE: 0.011976080946624279, time taken: 4.21s
Epoch 8 loss  MSE: 0.01204698346555233, time taken: 4.38s


KeyboardInterrupt: ignored

In [ ]:
model.summary()

Model: "our_model_bert_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             multiple                  16128     
_________________________________________________________________
dropout_80 (Dropout)         multiple                  0         
_________________________________________________________________
ffn_4 (FFN)                  multiple                  590851    
_________________________________________________________________
ffn_5 (FFN)                  multiple                  197121    
Total params: 804,100
Trainable params: 804,100
Non-trainable params: 0
_________________________________________________________________


## Get Bert Embeddings

In [ ]:
# # Generated here and saved once, because it gives OOM error when less space is avialable
# # Thus token_position_data and bert_emb are saved once

# bert = TFBertModel.from_pretrained("bert-base-cased")
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# # get maximum padding length
# tokenized_output_data = tokenizer.batch_encode_plus(
#             list(features['sentence']),
#             add_special_tokens=True, 
#             padding='longest', 
#             return_tensors="tf"
#         )
# max_len = tokenized_output_data['input_ids'].shape[1]

# sentences, tokens = list(features['sentence']), list(features['token'])

# cls_emb = []
# token_position_data = []
# token_emb = []

# #  get embeddings one by one
# for i in tqdm(range(len(sentences))):
#     tok = tokenizer.encode_plus(
#         sentences[i],
#         add_special_tokens=True, 
#         padding='max_length',
#         max_length = max_len,
#         return_tensors="tf"
#     )
#     entire = bert(**tok, return_dict=False)[0]
#     cls_emb.append(entire[:,0,:])
#     token_position_data.append( [ 
#                                     len(tokenizer.encode(sentences[i].split(tokens[i])[0], add_special_tokens=False)) + 1, 
#                                     len(tokenizer.encode(tokens[i], add_special_tokens=False))
#                                 ]
#                                )
#     token_emb.append(tf.reduce_mean(tf.gather(entire, tf.range(token_position_data[-1][0],token_position_data[-1][0]+token_position_data[-1][1]), axis=1), axis=1))

# cls_emb = np.squeeze(np.array(cls_emb), axis=1)
# token_emb = np.squeeze(np.array(token_emb), axis=1)
# token_position_data = np.array(token_position_data)

# print(cls_emb.shape)
# print(token_emb.shape)
# print(token_position_data.shape)

# np.save(FOLDER_PATH+"data/cls_emb.npy", cls_emb)
# np.save(FOLDER_PATH+"data/token_position_data_bert.npy", np.array(token_position_data))
# np.save(FOLDER_PATH+"data/token_emb_bert.npy", token_emb)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


## Load saved bert embeddings 

In [ ]:
cls_emb = np.load(FOLDER_PATH+"data/cls_emb.npy")
token_position_data = np.load(FOLDER_PATH+"data/token_position_data_bert.npy")
token_emb = np.load(FOLDER_PATH+"data/token_emb_bert.npy")

In [ ]:
sorted(token_position_data[:,0])[-15:]

[71, 73, 73, 75, 76, 78, 79, 79, 86, 87, 87, 97, 114, 231, 329]

## Bert code - 2

In [ ]:
# -------------------------------------------------------------------------------------
# feed fwd neural network :
# 1 hidden size layer, 1 output_size layer  and a dropout between them
# used for calculating weights as well as final complexity prediction 
#-------------------------------------------------------------------------------------

class FFN(tf.keras.layers.Layer):
    def __init__(
        self, 
        hidden_size,
        output_size,
        rate,
        softmax=False,
    ):
        super(FFN, self).__init__()

        self.layer1 = tf.keras.layers.Dense(hidden_size, activation="relu")  # (batch_size, hidden_size)
        self.dropout = tf.keras.layers.Dropout(rate)
        self.layer2 = tf.keras.layers.Dense(output_size, activation= "softmax" if softmax is True else None)  # (batch_size, output_size)

    def call(self, x, training):
        return self.layer2(self.dropout(self.layer1(x), training=training) ) 
        # return self.layer2(self.dropout(x, training=training)) 

# -------------------------------------------------------------------------------------
# Custom Model for predicting complexity, using BERT
# warning - for pooler gradients not being updated
# because we're not using the pooler, therefore it doesnt get updated
#-------------------------------------------------------------------------------------
class OurModelBert(tf.keras.Model):
    def __init__(
        self,
        bert_model,
        hidden_size,
        random_seed,
        rate=0.25
        ):
        """
        bert_model - model name or a path to saved model
        hidden_size - for FFN
        """

        super(OurModelBert, self).__init__()
        tf.random.set_seed(random_seed)
        self.random_seed = random_seed
        
        self.tokenizer = BertTokenizer.from_pretrained(bert_model)
        self.bert = TFBertModel.from_pretrained(bert_model)
        
        # make it non trainable
        self.bert.trainable = False
        # self.dense = tf.keras.layers.Dense(768, activation='relu')
        self.getWeights = FFN(hidden_size, 2, rate, True)
        self.final_layer = FFN(hidden_size, 1, rate)
    
    # one fwd pass on the model
    def call(self, inputs, token_position, feature_emb, training=False):
        """
        inputs : a dict of paramsters to Bert model with keys (input_ids, token_type_ids, attention_mask)
        token_position = [batch_size, 2] - [start,end] for tokens for which complexity is to be predicted
        """

        # bert pass : returns last hidden state, pooler output 
        
        # sentence_output = tf.concat(sentence_output,axis=0)
        sequence_output, _ = self.bert(**inputs, return_dict=False)       #(batch_size, seq_len, 768)

        # overall sentence embedding is CLS emb representation
        cls_emb = sequence_output[:,0,:]        # (batch_size, 768)

        # extract token embeddings    
        # for each example we have start and end token position
        # we add these embeddings position
        temp = []
        for s,t in zip(sequence_output, token_position):
            start,end = t[0],t[1]
            if (t[0]>self.max_length):
                start,end = 1,self.max_length
            elif (t[0]+t[1]>self.max_length):
                end = self.max_length

            temp.append(tf.reduce_sum(tf.gather(s, tf.range(start,start+end), axis=0), axis=0))
        token_emb = tf.stack(temp)
        # # OOM FOR LARGE INPUT
        # token_emb = tf.stack(
        #         [tf.reduce_sum(tf.gather(s, tf.range(t[0],t[0]+t[1]), axis=0), axis=0) for s,t in zip(sequence_output,token_position) ]
        #     )   # (batch_size, 768)

        # feature_emb = self.dense(feature_emb)
        # get weights
        weights = self.getWeights(tf.concat([token_emb, cls_emb], axis=1), training)    # (batch_size, 3)

        # generate attenton-based final embeddings
        # final_emb = weights[0]*token_emb + weights[1]*added_emb
        final_emb = tf.zeros(shape=(), dtype=tf.dtypes.float32)   # (bacth_size, 768)
        final_emb += tf.expand_dims(weights[:, 0], axis=1) * token_emb
        final_emb += tf.expand_dims(weights[:, 1], axis=1) * cls_emb
        # final_emb += tf.expand_dims(weights[:, 2], axis=1) * feature_emb

        # output complexity
        final_output = self.final_layer(final_emb)

        return final_output

    # loss function
    def loss_function(self, real, pred):
        loss_ = tf.keras.losses.MSE(real, pred)
        l = tf.reduce_mean(loss_)
        return l

    # set optimizer
    def compile(self, optimizer):
        super(OurModelBert, self).compile()
        self.optimizer = optimizer

    # train step - fwd pass + back prop + update model weights
    # @tf.function() - may not work beacuse zip is used
    def train_step(self, inputs, token_position, y, feature_emb):
        """
        sentences: batch wise sentences
        inputs : a dict of paramsters to Bert model with keys (input_ids, token_type_ids, attention_mask, training, return_dict)
        token_position = [batch_size, 2] - [start,end] for tokens for which complexity is to be predicted
        """

        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self(inputs, token_position, feature_emb, True)  
            loss = self.loss_function(y, y_pred)

        # calculate gradients
        gradients = tape.gradient(loss, self.trainable_variables)

        # update model weights using gradients
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return loss

    # fit funtion of model
    def _fit(
        self, 
        sentences, 
        tokens, 
        feature_emb,
        complexity,
        epochs, 
        batch_size,
        val_sentences=None,
        val_tokens=None,
        val_feature_emb=None,
        val_complexity=None
        ):
        """
        sentences, tokens, complexity - from dataset -  a list is expected
        """

        # tokenize + pad data
        # tokenized_output = dict of input_ids, token_type_ids, attention_mask
        tokenized_output_data = self.tokenizer.batch_encode_plus(
            sentences,
            add_special_tokens=True, 
            padding='max_length',
            max_length=50, 
            return_tensors="tf",
            truncation=True,
        )
        # self.max_length=tokenized_output_data['input_ids'].shape[1]
        self.max_length = 50

        # identify token position
        # token_position_data is a list of [start, length] for all sentences
        # [start, end] = [start, start+length-1]  => will be done in call function
        # find the token in sentence and 1 is added to accomodate CLS token

        token_position_data =   [ 
                                    [ 
                                        len(self.tokenizer.encode(s.split(t)[0], add_special_tokens=False)) + 1, 
                                        len(self.tokenizer.encode(t, add_special_tokens=False))
                                    ]
                                    for s,t in zip(sentences, tokens) 
                                ]

        # create tensorflow dataset
        tf_data = tf.data.Dataset.from_tensor_slices((tokenized_output_data, token_position_data, feature_emb, complexity))

        # shuffle and batch
        tf_data = tf_data.shuffle(100000, seed=self.random_seed).batch(batch_size)

        train_loss = tf.keras.metrics.Mean(name="train_loss")
        loss_train = {"MSE": []}
        loss_val = {"MSE": []}

        #training starts
        for epoch in range(epochs):
            start = time.time()
            
            # reset state for mean loss
            train_loss.reset_states()

            # loop over batches
            for step, x in tqdm(tf_data.enumerate()):

                # get inputs for model
                tokenized_output = x[0]
                token_position = x[1]
                feature_emb=x[2]
                y = x[3]

                # call trainstep
                loss = self.train_step(tokenized_output, token_position, y, feature_emb)
                train_loss(loss)

            loss_train["MSE"].append(train_loss.result().numpy())
            print("Epoch {} loss  MSE: {}, time taken: {:.2f}s".format(epoch + 1, loss_train["MSE"][-1], time.time() - start))

            # validation if provided
            if (val_sentences!=None):
                val_pred = self._predict(val_sentences, val_tokens, val_feature_emb)
                loss_val["MSE"].append(self.loss_function(val_complexity, val_pred).numpy())
                print("Validation loss MSE : {}".format(loss_val["MSE"][-1]))
        
        if (val_sentences!=None):
            return loss_train, loss_val
        return loss_train

    
    # predict function
    def _predict(self, sentences, tokens, feature_emb):

        # tokenize + pad data - only till max length
        # tokenized_output = dict of input_ids, token_type_ids, attention_mask
        tokenized_output_data = self.tokenizer.batch_encode_plus(
            sentences,
            add_special_tokens=True, 
            padding='max_length',
            max_length=self.max_length,
            truncation=True,
            return_tensors="tf"
        )

        token_position_data =   [ 
                                    [ 
                                        len(self.tokenizer.encode(s.split(t)[0], add_special_tokens=False)) + 1, 
                                        len(self.tokenizer.encode(t, add_special_tokens=False))
                                    ]
                                    for s,t in zip(sentences, tokens) 
                                ]

        # create tensorflow dataset
        tf_data = tf.data.Dataset.from_tensor_slices((tokenized_output_data,token_position_data, feature_emb))

        # batch for memory constraints
        tf_data = tf_data.batch(32)

        pred_list = []
        for step, x in tf_data.enumerate():
            # get inputs for model
            input_seq = x[0]
            token_position = x[1]
            feature_emb = x[2]

            pred_list.append(self(input_seq, token_position, feature_emb))
        
        return tf.concat(pred_list,axis=0)

In [ ]:
print(len(sentences_train_list))
print(len(sentences_test_list))

print(len(tokens_train_list))
print(len(tokens_test_list))

features_train = train[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
                        'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
                        'google frequency1', 'google frequency2', 
                        'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values

features_test = test[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
                        'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
                        'google frequency1', 'google frequency2', 
                        'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values

print("feature emb shape train : {}".format(features_train.shape))
print("feature emb shape test : {}".format(features_test.shape))

8261
918
8261
918
feature emb shape train : (8261, 20)
feature emb shape test : (918, 20)


In [ ]:
PRETRAINED_BERT_MODEL = "bert-base-cased"
model = OurModelBert(bert_model=PRETRAINED_BERT_MODEL, hidden_size=10, random_seed=42, rate=0.3)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
model._fit(sentences_train_list, tokens_train_list, features_train, complexity_train_list, 20, 32, sentences_test_list, tokens_test_list, features_test, complexity_test_list)


Epoch 1 loss  MSE: 0.20865324139595032, time taken: 197.05s
Validation loss MSE : 0.02044394612312317



Epoch 2 loss  MSE: 0.04495782405138016, time taken: 195.82s
Validation loss MSE : 0.020448489114642143



Epoch 3 loss  MSE: 0.02836226113140583, time taken: 195.51s
Validation loss MSE : 0.020445793867111206



Epoch 4 loss  MSE: 0.022957809269428253, time taken: 195.51s
Validation loss MSE : 0.020444029942154884


In [ ]:
model.summary()

## Evaluate

In [ ]:
print(len(list(test_features['sentence'])))
print(len(list(test_features['token'])))

features = test_features[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
                        'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
                        'google frequency1', 'google frequency2', 
                        'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values

print("feature shape : {}".format(features.shape))

SUBMISSION_FOLDER = os.path.join(FOLDER_PATH,"predictions/approach1/bert/single")

if( not os.path.exists(SUBMISSION_FOLDER)):
    os.makedirs(SUBMISSION_FOLDER)

# predict 
pred = model._predict(sentences, tokens, feature_emb)
pred = pd.DataFrame({"ID":test_features.index, "complexity":pred.numpy().reshape(-1,)})
pred.to_csv(SUBMISSION_FOLDER+"/app_bert1.csv", index=False, header=False)

evaluate(SUBMISSION_FOLDER, FOLDER_PATH+"/references/lcp_single_test_labelled_preprocessed.csv")


For file app1.csv
pearson  :  0.6566982077355423
spearman :  0.6628609892192908
mae      :  0.08108016534811723
mse      :  0.011103258259959664
r2       :  0.3139816796663809


In [ ]:
SUBMISSION_FOLDER = os.path.join(FOLDER_PATH,"predictions/approach1/multi")

if( not os.path.exists(SUBMISSION_FOLDER)):
    os.makedirs(SUBMISSION_FOLDER)

# predict 
pred = model._predict(multi_sent_emb, multi_token_data, features_multi)
pred = pd.DataFrame({"ID":test_multi_features.index, "complexity":pred.numpy().reshape(-1,)})
pred.to_csv(SUBMISSION_FOLDER+"/app1.csv", index=False, header=False)


evaluate(SUBMISSION_FOLDER, FOLDER_PATH+"/references/lcp_multi_test_labelled_preprocessed.csv")


For file app1.csv
pearson  :  0.7515866884564054
spearman :  0.7702903717771301
mae      :  0.10772707952766654
mse      :  0.017540098451649166
r2       :  0.2733147271433434


## test bert code

In [ ]:
# ----------- test model output shape

sample_model = OurModelBert( bert_model="bert-base-cased",
        hidden_size=100,
        random_seed=12,
        )

inputs = sample_model.tokenizer.batch_encode_plus(["this is a senten ce", "dada af"], add_special_tokens=True, padding='max_length', max_length=20, return_tensors="tf")
token_position_data = tf.constant([(3,5) for _ in range(2)])

output = sample_model(inputs, token_position_data)
print(output.shape)

#  ----------- test fit and predict

sample_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

sentences = ["this is a senten ce", "dada af", "third sentence is this ok"]
tokens =  ["senten", "a", "this ok"]

y = tf.random.uniform((3, 1), dtype=tf.float32, minval=0, maxval=1)

loss = sample_model._fit( 
    sentences=sentences, 
    tokens=tokens, 
    complexity=y,
    epochs = 10, 
    batch_size = 32,
    )


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


(2, 1)
<TensorSliceDataset shapes: ({input_ids: (9,), token_type_ids: (9,), attention_mask: (9,)}, (2,), (1,)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int32, tf.float32)>



Epoch 1 loss  MSE: 0.3736450672149658, time taken: 0.54s



Epoch 2 loss  MSE: 0.3736450672149658, time taken: 0.38s



Epoch 3 loss  MSE: 0.3736450672149658, time taken: 0.34s


...


Epoch 6 loss  MSE: 0.3736450672149658, time taken: 0.37s



Epoch 7 loss  MSE: 0.3736450672149658, time taken: 0.41s



Epoch 8 loss  MSE: 0.3736450672149658, time taken: 0.40s



Epoch 9 loss  MSE: 0.3736450672149658, time taken: 0.55s



Epoch 10 loss  MSE: 0.3736450672149658, time taken: 0.35s


In [ ]:
FOLDER_PATH = "/content/drive/MyDrive/CS60075-Team28-Task-1"
DATA_FOLDER = FOLDER_PATH+"/data/preprocessed"

In [ ]:
df = pd.read_csv(os.path.join(DATA_FOLDER, "lcp_single_train_preprocessed.csv"), index_col=0)
df.head()

,corpus,sentence,token,complexity
id,,,,
3ZLW647WALVGE8EBR50EGUBPU4P32A,bible,behold came river seven cattle sleek fat fed m...,river,0.000000
34R0BODSP1ZBN3DVY8J8XSIY551E5C,bible,fellow bondservant brother prophet keep word book,brother,0.000000
3S1WOPCJFGTJU2SGNAN2Y213N6WJE3,bible,man lord land said u know honest men leave one...,brother,0.050000
3BFNCI9LYKQN09BHXHH9CLSX5KP738,bible,shimei sixteen son six daughter brother didnt ...,brother,0.150000
3G5RUKN2EC3YIWSKUXZ8ZVH95R49N2,bible,put brother far,brother,0.263889


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df['sentence'] = df['sentence'].astype(str)
df['token'] = df['token'].astype(str)

sentences = list(df["sentence"])
tokens = list(df["token"])
complexity = list(df["complexity"])


In [ ]:
model = OurModelBert(bert_model=PRETRAINED_BERT_MODEL, hidden_size=10, random_seed=42)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001))
model._fit(sentences, tokens, complexity, 10, 64)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


<TensorSliceDataset shapes: ({input_ids: (331,), token_type_ids: (331,), attention_mask: (331,)}, (2,), ()), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int32, tf.float32)>


ResourceExhaustedError: ignored

In [ ]:
model.summary()

Model: "our_model_bert_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_bert_model_19 (TFBertMode multiple                  108310272 
_________________________________________________________________


ValueError: ignored

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

tok = tokenizer.encode_plus("this is a senten ce", add_special_tokens=True, padding='max_length', max_length=20, return_tensors="tf")
print(tok)

# model = TFBertModel.from_pretrained(PRETRAINED_BERT_MODEL)
# last_hidden_state,pooler_output = model(**tok, return_dict=False)
# cls = last_hidden_state[:,0,:]

{'input_ids': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=
array([[ 101, 1142, 1110,  170, 1850, 1424,  172, 1162,  102,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>}


In [ ]:
sentences = ["this is a senten ce"]
tokens = ["senten"]

In [ ]:
("his is a senten ce").split(tokens[0])

['his is a ', ' ce']

In [ ]:
tokenizer.encode(sentences[0].split(tokens[0])[0], add_special_tokens=False)

[1142, 1110, 170]

In [ ]:
token_position_data =   [ 
                                    [ 
                                        len(s.split(t)[0].split()), 
                                        len(t.split())
                                    ]
                                    for s,t in zip(sentences, tokens) 
                                ]

In [ ]:
token_position_data

[[3, 1]]

In [ ]:

# Dataset = namedtuple('Dataset', 'name, train, test')
# Model = namedtuple('Model', 'type, name, dimension, corpus, model')


# from gensim.test.utils import datapath, get_tmpfile
# from gensim.models import KeyedVectors
# from gensim.scripts.glove2word2vec import glove2word2vec

# MAIN_PATH = 'embeddings/'

# glove_models = []

# glove_defs = [ Model('glove', 'glove.6B.300d.txt', 300, 'wikipedia+gigaword5', None)]
              
# for model in glove_defs:
#     glove_file = MAIN_PATH + model.name
#     tmp_file = get_tmpfile(model.name + '-temp')
#     glove2word2vec(glove_file, tmp_file)
#     vecs = KeyedVectors.load_word2vec_format(tmp_file)
#     glove_models.append(Model(model.type, model.name, model.dimension, model.corpus, vecs))
#     print('load model : {}'.format(model.name))
    
# print(glove_models)

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(sentences_train_list)
# words_to_index = tokenizer.word_index
# len(words_to_index)

# vocab_len = len(words_to_index)
# embed_vector_len = word_to_vec_map['hello'].shape[0]

# emb_matrix = np.zeros((vocab_len+1, embed_vector_len))

# for word, index in words_to_index.items():
#   embedding_vector = word_to_vec_map.get(word)
#   if embedding_vector is not None:
#     emb_matrix[index, :] = embedding_vector

# embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=256, weights = [emb_matrix], trainable=False)

# emb_matrix.shape
